### 1. Graph

In [2]:
import tensorflow as tf
import timeit
from datetime import datetime

In [45]:
def function_to_get_faster(x, y, b):
    x = tf.matmul(x, y)
    x = x+b
    return x

a_function_that_uses_a_graph = tf.function(function_to_get_faster)

x1 = tf.constant([[1.0,2.0]])
y1 = tf.constant([[2.0],[3.0]])
b1 = tf.constant(4.0)

a_function_that_uses_a_graph(x1, y1, b1).numpy()

array([[12.]], dtype=float32)

In [44]:
def my_function(x):
    if tf.reduce_sum(x) <= 1:
        return x*x
    else:
        return x-1

a_function = tf.function(my_function)

print("First branch, with graph:", a_function(tf.constant(1.0)).numpy())
print("Second branch, with graph:", a_function(tf.constant([5.0, 5.0])).numpy())

First branch, with graph: 1.0
Second branch, with graph: [4. 4.]


In [50]:
%reload_ext tensorboard

def inner_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# Use the decorator
@tf.function
def outer_function(x):
  y = tf.constant([[2.0], [3.0]])
  b = tf.constant(4.0)

  return inner_function(x, y, b)

### 2. 层

In [28]:
class Dense(tf.Module):
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(tf.random.normal([in_features, out_features]),name='w')
        self.b = tf.Variable(tf.zeros([out_features]),name='b')
    def __call__(self,x):
        y = tf.matmul(self.w, x)+self.b
        return tf.nn.sigmoid(y)

In [19]:
my_module=Dense(in_features=3, out_features=3,name='mymodule')
my_module(tf.constant([[0.0288],[-0.3256],[0.5925]]))

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0.7431395 , 0.7431395 , 0.7431395 ],
       [0.30271006, 0.30271006, 0.30271006],
       [0.56767285, 0.56767285, 0.56767285]], dtype=float32)>

### 3. 从层到模型

In [39]:
class SequentialModel(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    
    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=1)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# make a model
my_model = SequentialModel(name="the_model")

# print("Model results: ", my_model(tf.constant([0.0288,-0.3256,0.5925])))

### 3. 模型存储与恢复

In [30]:
chkp_path = "my_checkpoint"
checkpoint = tf.train.Checkpoint(model=my_model)
checkpoint.write(chkp_path)

'my_checkpoint'

In [31]:
tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/dense_1/b/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('model/dense_1/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 3]),
 ('model/dense_2/b/.ATTRIBUTES/VARIABLE_VALUE', [1]),
 ('model/dense_2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 1])]

In [33]:
%tensorboard --logdir logs/func

Reusing TensorBoard on port 6006 (pid 11156), started 0:04:45 ago. (Use '!kill 11156' to kill it.)

In [40]:
#set up logging
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s'%stamp
writer = tf.summary.create_file_writer(logdir)

new_model = SequentialModel()

tf.summary.trace_on(graph=True, profiler=True)

outer_function(tf.constant([[1.0,2.0]]))
with writer.as_default():
    tf.summary.trace_export(
        name="my_func_trace",
        step=0,
        profiler_outdir=logdir)

In [41]:
tf.saved_model.save(my_model, 'the_saved_model')

INFO:tensorflow:Assets written to: the_saved_model\assets


In [43]:
$ ls -l the_saved_model

SyntaxError: invalid syntax (<ipython-input-43-6f0a282edb47>, line 1)

### 4. 训练

In [52]:
TRUE_W = 3.0
TRUE_B = 2.0

NUM_EXAMPLES = 1000

x = tf.random.normal(shape=[NUM_EXAMPLES])

noise = tf.random.normal(shape=[NUM_EXAMPLES])

y = x*TRUE_W+TRUE_B+noise

In [54]:
# Plot all the data
import matplotlib.pyplot as plt

plt.scatter(x, y, c="b")
plt.show()

SyntaxError: invalid syntax (<ipython-input-54-6f7808bf4b2d>, line 2)

In [55]:
class MyModel(tf.Module):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    # Initialize the weights to `5.0` and the bias to `0.0`
    # In practice, these should be randomly initialized
    self.w = tf.Variable(5.0)
    self.b = tf.Variable(0.0)

  def __call__(self, x):
    return self.w * x + self.b

model = MyModel()

# List the variables tf.modules's built-in variable aggregation.
print("Variables:", model.variables)

# Verify the model works
assert model(3.0).numpy() == 15.0

Variables: (<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>, <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>)


In [56]:
# This computes a single loss value for an entire batch
def loss(target_y, predicted_y):
  return tf.reduce_mean(tf.square(target_y - predicted_y))

In [57]:
# Given a callable model, inputs, outputs, and a learning rate...
def train(model, x, y, learning_rate):

  with tf.GradientTape() as t:
    # Trainable variables are automatically tracked by GradientTape
    current_loss = loss(y, model(x))

  # Use GradientTape to calculate the gradients with respect to W and b
  dw, db = t.gradient(current_loss, [model.w, model.b])

  # Subtract the gradient scaled by the learning rate
  model.w.assign_sub(learning_rate * dw)
  model.b.assign_sub(learning_rate * db)

In [58]:
model = MyModel()

# Collect the history of W-values and b-values to plot later
Ws, bs = [], []
epochs = range(10)

# Define a training loop
def training_loop(model, x, y):

  for epoch in epochs:
    # Update the model with the single giant batch
    train(model, x, y, learning_rate=0.1)

    # Track this before I update
    Ws.append(model.w.numpy())
    bs.append(model.b.numpy())
    current_loss = loss(y, model(x))

    print("Epoch %2d: W=%1.2f b=%1.2f, loss=%2.5f" %
          (epoch, Ws[-1], bs[-1], current_loss))


In [ ]:
print("Starting: W=%1.2f b=%1.2f, loss=%2.5f" %
      (model.w, model.b, loss(y, model(x))))

# Do the training
training_loop(model, x, y)

# Plot it
plt.plot(epochs, Ws, "r",
         epochs, bs, "b")

plt.plot([TRUE_W] * len(epochs), "r--",
         [TRUE_B] * len(epochs), "b--")

plt.legend(["W", "b", "True W", "True b"])
plt.show()
